# Upscale testing

In [ ]:
from PIL import Image
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import imageio.v3 as iio
import matplotlib.pyplot as plt
import numpy as np
from helper_functions.helper_functions import pixels_between_points
import pathlib
import platform
plt2 = platform.system()
if plt2 == 'Windows': pathlib.PosixPath = pathlib.WindowsPath

from scipy.ndimage import map_coordinates

: 

In [ ]:
test_image = np.zeros((10,10))
# Pixels
test_image[2, 2:8] = 1
test_image[7, 2:8] = 1
test_image[2:7, 2] = 1
test_image[2:7, 7] = 1
print(test_image)
# 400 points
x_top = np.linspace(2,7)
y_top = np.array([2 for _ in range(len(x_top))])
x_bottom = np.linspace(7,2)
y_bottom = np.array([7 for _ in range(len(x_top))])

x_right = np.array([7 for _ in range(len(x_top))])
y_right = np.linspace(2,7)

x_left = np.array([2 for _ in range(len(x_top))])
y_left = np.linspace(7,2)

fourier_xs = np.concatenate(( x_left, x_top, x_right, x_bottom,)) +0.001
fourier_ys = np.concatenate(( y_left, y_top, y_right, y_bottom,)) +0.001

# Define the scaling factor 
scale_factor_x = 4
scale_factor_y = 8

fourier_xs_with_offset = fourier_xs*scale_factor_x+scale_factor_x/2-0.5
fourier_ys_with_offset = fourier_ys*scale_factor_y+scale_factor_y/2-0.5

: 



Use the get_pixels_between() on the upscaled image using different border scaled differently. Plot and print image to see which pixels are grabbed.
Visually verify the new pixel border an see if it lines up correctly in the printout.

In [ ]:
def gridlines(image: np.ndarray):
    xs = []
    ys = []
    for x in range(image.shape[1]):
        # x=0
        xs += [x-0.5, x-0.5, None]
        ys += [-0.5, image.shape[0]-0.5, None]
    for x in range(image.shape[0]):
        # y=0
        ys += [x-0.5, x-0.5, None]
        xs += [-0.5, image.shape[1]-0.5, None]
    return xs,ys

: 

# Proving the offset is necessary

Granule explorer uses `map_coordinates()` from scipy on granule images. This method treats pixel coordinates as their 'center'. Therfore we need to add the offset when scaling the border.
Write about this in the thesis. Could not find any information about this on the internet??

In [ ]:
# map_coordinates() uses pixels as integer centered. This means any value sampled +- 1/2 from the integer coordinate will be the same as the same.
# Had map_coordinates() instead treated them as 'corner integered' a different value would have been returned when we deviate from the coordinate in the minus direction.
assert map_coordinates(test_image, [[1.9],[1.9]], order=0) == np.array([1]), "At this position the value is expected to be 1."
assert map_coordinates(test_image, [[0],[0]], order=0) == np.array([0]), "At this position the value is expected to be 0."

: 

In [ ]:
# original_image = Image.fromarray(test_image)

# # Get the size of the original image
# original_width, original_height = original_image.size
# # Calculate the new size of the image after upscaling
# new_width = original_width * scale_factor
# new_height = original_height * scale_factor
# # Resize the image using the new size
# upscaled_image_box = np.array(original_image.resize((new_width, new_height), Image.Resampling.NEAREST))        

# fig = make_subplots(rows=1, cols=1, 
#                     horizontal_spacing=0.01, 
#                     vertical_spacing=0.1,
#                     subplot_titles=('Original image', f"Upscaled image s={scale_factor}"))
# # Add images to plot
# base_image_fig = px.imshow(test_image)
# fig.add_trace(base_image_fig.data[0], 1, 1)
# xs, ys = gridlines(test_image)
# fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=1)

# zi = map_coordinates(original_image, [[1.9],[1.9]], order=0)
# print(zi)
# fig.add_trace(go.Scatter(x=[2,2], y=[7,7], marker=dict(color='red', size=10), name=f"Point"), row=1, col=1)

# fig.update_layout(title_text=f"", title_x=0.5, width=1400, height=800, showlegend=True, font_size=11)
# fig.update(layout_coloraxis_showscale=False)
# # fig.show()

: 

In [ ]:
original_image = Image.fromarray(test_image)

# Get the size of the original image
original_width, original_height = original_image.size
# Calculate the new size of the image after upscaling
new_width = original_width * scale_factor_x
new_height = original_height * scale_factor_y
# Resize the image using the new size
upscaled_image_box = np.array(original_image.resize((new_width, new_height), Image.Resampling.NEAREST))        

fig = make_subplots(rows=1, cols=2, 
                    horizontal_spacing=0.01, 
                    vertical_spacing=0.1,
                    subplot_titles=('Original image', f"Upscaled image s={scale_factor_x, scale_factor_y}"))
# Add images to plot
base_image_fig = px.imshow(test_image)
fig.add_trace(base_image_fig.data[0], 1, 1)
xs, ys = gridlines(test_image)
fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=1)

upscaled_image_fig = px.imshow(upscaled_image_box)
fig.add_trace(upscaled_image_fig.data[0], 1, 2)
xs, ys = gridlines(upscaled_image_box)
fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=2)


fig.add_trace(go.Scatter(x=[2,2], y=[7,7], marker=dict(color='red', size=10), name=f"Point"), row=1, col=1)
fig.add_trace(go.Scatter(x=[2*scale_factor_x+scale_factor_x/2-0.5,2*scale_factor_x+scale_factor_x/2-0.5], y=[7*scale_factor_y+scale_factor_y/2-0.5,7*scale_factor_y+scale_factor_y/2-0.5], marker=dict(color='red', size=10), name=f"Point"), row=1, col=2)

fig.update_layout(title_text=f"", title_x=0.5, width=1400, height=800, showlegend=True, font_size=11)
fig.update(layout_coloraxis_showscale=False)
fig.show()

: 

In [ ]:
fig = make_subplots(rows=1, cols=2, 
                    horizontal_spacing=0.05, 
                    vertical_spacing=0.1,
                    subplot_titles=('Original image', f"Upscaled image s={scale_factor_x, scale_factor_y}"))
# Add images to plot
base_image_fig = px.imshow(test_image)
fig.add_trace(base_image_fig.data[0], 1, 1)
xs, ys = gridlines(test_image)
fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=1)

upscaled_image_fig = px.imshow(upscaled_image_box)
fig.add_trace(upscaled_image_fig.data[0], 1, 2)
xs, ys = gridlines(upscaled_image_box)
fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=2)


fig.add_trace(go.Scatter(x=fourier_xs, y=fourier_ys, marker=dict(color='red', size=10), name=f"400p border"), row=1, col=1)
fig.add_trace(go.Scatter(x=fourier_xs*scale_factor_x, y=fourier_ys*scale_factor_y, marker=dict(color='red', size=10), name=f"400p border "), row=1, col=2)

fig.update_layout(title_text=f"Border upscaling when b(point)=point*s", title_x=0.5, width=1400, height=800, showlegend=True, font_size=11)
fig.update(layout_coloraxis_showscale=False)
fig.show()

: 

In [ ]:
fig = make_subplots(rows=1, cols=2, 
                    horizontal_spacing=0.05, 
                    vertical_spacing=0.1,
                    subplot_titles=('Original image', f"Upscaled image s={scale_factor_x, scale_factor_y}"))
# Add images to plot
base_image_fig = px.imshow(test_image)
fig.add_trace(base_image_fig.data[0], 1, 1)
xs, ys = gridlines(test_image)
fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=1)

upscaled_image_fig = px.imshow(upscaled_image_box)
fig.add_trace(upscaled_image_fig.data[0], 1, 2)
xs, ys = gridlines(upscaled_image_box)
fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=2)

# 400p borders
fig.add_trace(go.Scatter(x=fourier_xs, y=fourier_ys, marker=dict(color='red', size=10), name=f"400p border"), row=1, col=1)
fig.add_trace(go.Scatter(x=fourier_xs*scale_factor_x+scale_factor_x/2-0.5, y=fourier_ys*scale_factor_y+scale_factor_y/2-0.5, marker=dict(color='red', size=10), name=f"400 p border"), row=1, col=2)

fig.update_layout(title_text=f"Border upscaling with offset: b(point)=point*s + s/2 - 1/2", title_x=0.5, width=1400, height=800, showlegend=True, font_size=11)
fig.update(layout_coloraxis_showscale=False)
fig.show()

: 

# Get intersected pixels

In [ ]:
fig = make_subplots(rows=1, cols=2, 
                    horizontal_spacing=0.05, 
                    vertical_spacing=0.1,
                    subplot_titles=('Original image', f"Upscaled image s={scale_factor_x, scale_factor_y}"))
# Add images to plot
base_image_fig = px.imshow(test_image)
fig.add_trace(base_image_fig.data[0], 1, 1)
xs, ys = gridlines(test_image)
fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=1)

upscaled_image_box_2 = upscaled_image_box.copy()
xs, ys = pixels_between_points(fourier_xs*scale_factor_x, fourier_ys*scale_factor_y)
# xs, ys = pixels_between_points(fourier_xs*scale_factor+scale_factor/2-0.5, fourier_ys*scale_factor+scale_factor/2-0.5)
for x,y in list(zip(xs,ys)):
    upscaled_image_box_2[int(y)][int(x)] = 0.5
    
upscaled_image_fig = px.imshow(upscaled_image_box_2)
fig.add_trace(upscaled_image_fig.data[0], 1, 2)
xs, ys = gridlines(upscaled_image_box_2)
fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=2)

# 400p borders
fig.add_trace(go.Scatter(x=fourier_xs, y=fourier_ys, marker=dict(color='red', size=10), name=f"400p border"), row=1, col=1)
fig.add_trace(go.Scatter(x=fourier_xs*scale_factor_x, y=fourier_ys*scale_factor_y, marker=dict(color='red', size=10), name=f"400 p border"), row=1, col=2)

fig.update_layout(title_text=f"Border upscaling with no offset: b(point)=point*s", title_x=0.5, width=1400, height=800, showlegend=True, font_size=11)
fig.update(layout_coloraxis_showscale=False)
fig.show()

: 

# 

scipy.ndimage import map_cooridnates

In [ ]:
fig = make_subplots(rows=1, cols=2, 
                    horizontal_spacing=0.05, 
                    vertical_spacing=0.1,
                    subplot_titles=('Original image', f"Upscaled image s={scale_factor_x}"))
# Add images to plot
base_image_fig = px.imshow(test_image)
fig.add_trace(base_image_fig.data[0], 1, 1)
xs, ys = gridlines(test_image)
fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=1)

upscaled_image_box_3 = upscaled_image_box.copy()
xs_pixel, ys_pixel = pixels_between_points(fourier_xs_with_offset, fourier_ys_with_offset)
# xs, ys = pixels_between_points(fourier_xs*scale_factor+scale_factor/2-0.5, fourier_ys*scale_factor+scale_factor/2-0.5)
for x,y in list(zip(xs_pixel,ys_pixel)):
    upscaled_image_box_3[int(y)][int(x)] = 0.5
    
upscaled_image_fig = px.imshow(upscaled_image_box_3)
fig.add_trace(upscaled_image_fig.data[0], 1, 2)
xs, ys = gridlines(upscaled_image_box_3)
fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=2)

# 400p borders
fig.add_trace(go.Scatter(x=fourier_xs, y=fourier_ys, marker=dict(color='red', size=10), name=f"400p border"), row=1, col=1)
fig.add_trace(go.Scatter(x=fourier_xs_with_offset, y=fourier_ys_with_offset, marker=dict(color='red', size=10), name=f"400 p border"), row=1, col=2)

fig.update_layout(title_text=f"Border upscaling with offset: b(point)=point*s+s/2 - 1/2", title_x=0.5, width=1400, height=800, showlegend=True, font_size=11)
fig.update(layout_coloraxis_showscale=False)
fig.show()

: 

In [ ]:
# print(test_image)
# print(upscaled_image_box)

: 

In [ ]:
fig = make_subplots(rows=1, cols=2, 
                    horizontal_spacing=0.05, 
                    vertical_spacing=0.1,
                    subplot_titles=('Original image', f"Upscaled image s={scale_factor_x, scale_factor_y}"))
# Add images to plot
base_image_fig = px.imshow(test_image)
fig.add_trace(base_image_fig.data[0], 1, 1)
xs, ys = gridlines(test_image)
fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=1)

upscaled_image_box_3 = upscaled_image_box.copy()
test_x,test_y = np.array([13,19.3]), np.array([19.6,13]) # TODO: Create better implementation. Everything works now? Note the +1/2 to
# TODO: pixels_between_points_2() needs to be pushed by 1/2 to align with 'center pixel' method used by python libraries.
xs_pixel, ys_pixel = pixels_between_points(test_x,test_y) # When given coordinates to this function, add 1/2 to treat pixels as 'corner integer position' and not 'center integered'.
fig.add_trace(go.Scatter(x=test_x, y=test_y, marker=dict(color='cyan', size=10), name=f"Test line"), row=1, col=2)
# xs_pixel, ys_pixel = pixels_between_points_2(fourier_xs*scale_factor, fourier_ys*scale_factor)
for y,x in list(zip(xs_pixel,ys_pixel)):
    try:
        upscaled_image_box_3[int(x)][int(y)] = 0.5 # x goes up, y goes right
    except:
        print("what")
        continue

upscaled_image_fig = px.imshow(upscaled_image_box_3)
fig.add_trace(upscaled_image_fig.data[0], 1, 2)
xs, ys = gridlines(upscaled_image_box_3)
fig.add_trace(go.Scatter(x=xs, y=ys, marker=dict(color='black', size=1), name=f"Pixel edges", showlegend=False), row=1, col=2)

# 400p borders
fig.add_trace(go.Scatter(x=fourier_xs, y=fourier_ys, marker=dict(color='red', size=10), name=f"400p border"), row=1, col=1)
fig.add_trace(go.Scatter(x=fourier_xs*scale_factor_x, y=fourier_ys*scale_factor_y, marker=dict(color='red', size=10), name=f"400p border"), row=1, col=2)
# fig.add_trace(go.Scatter(x=fourier_xs_with_offset, y=fourier_ys_with_offset, marker=dict(color='red', size=10), name=f"400 p border"), row=1, col=2)

fig.update_layout(title_text=f"Border upscaling with offset: b(point)=point*s+s/2 - 1/2", title_x=0.5, width=1400, height=800, showlegend=True, font_size=11)
fig.update(layout_coloraxis_showscale=False)
fig.show()

: 

In [ ]:
def print_matrix(matrix):
    with open('test.txt', 'w+') as f:
        for row in matrix:
            l = ""
            for number in row:
                l += f" {int(number)} "
            f.write(l+"\n")

: 

In [ ]:
upscaled_image_box_4 = upscaled_image_box.copy()
# xs_pixel, ys_pixel = pixels_between_points_2(test_x,test_y)
xs_pixel, ys_pixel = pixels_between_points(fourier_xs*scale_factor_x+scale_factor_y/2-1/2, fourier_ys*scale_factor_y+scale_factor_y/2-1/2)
# fig.add_trace(go.Scatter(x=test_x,y=test_y, marker=dict(color='red', size=10), name=f"Test line"), row=1, col=2)
# xs_pixel, ys_pixel = pixels_between_points_2(fourier_xs*scale_factor, fourier_ys*scale_factor)
for x,y in list(zip(xs_pixel,ys_pixel)):
    try:
        upscaled_image_box_4[int(x)][int(y)] = 7 # x goes up, y goes right
    except:
        continue

print_matrix(upscaled_image_box_4)
# print_matrix(test_image)
# with open('test.txt', 'w+') as f:
#     # for row in upscaled_image_box_4:
#     f.writelines(map(str, upscaled_image_box_4))

: 

In [ ]:
fig = make_subplots(rows=1, cols=1, 
                    horizontal_spacing=0.05, 
                    vertical_spacing=0.1)
base_image_fig = px.imshow(test_image)
fig.add_trace(base_image_fig.data[0], 1, 1)
fig.update_layout(title_text=f"", title_x=0.5, width=700, height=700, showlegend=True, font_size=11)
fig.update(layout_coloraxis_showscale=False)
fig.show()

: 